# RSS Infomation Research

In [1]:
# RSS에서 xml태그 별 정보 추출
!pip install feedparser 

# 인터넷 신문기사 분석(라이브러리 내의 "Article()"사용하기 위해)
!pip install newspaper3k 

# 한국어 형태소 분석기 (주어진 문장에서 명사만 추출할 목적으로 사용)
!pip install konlpy 

# 라이브러리 불러오기

In [11]:
import feedparser # xml 태그별 정보 추출
from newspaper import Article # 인터넷 신문 기사 분석
from konlpy.tag import Okt # 한국어 형태소 분석기
from collections import Counter # 갯수 세기(TF계산용)
from operator import eq # 동일한 단어인지 여부 확인 위해 사용
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html 태그 삭제를 위해 사용

# 정보 수집할 rss 링크 설정 및 정보 수집하기

##### 전자신문 (https://www.etnews.com/rss/) 의 오늘의 뉴스 부분 

##### 함수 정의

In [2]:
# 제목, 링크 수집 함수
def crawl_rss(urls):
    array_rss = [] # 모든 기사를 넣을 리스트
    titles_rss = set() # 중복된 기사를 제거하기 위한 기사 제목들 집합
    
    for url in urls: # url 순회
        print(f"[Crawl Rss] {url}") # 현재 크롤링하는 rss 링크 출력
        parser_rss = feedparser.parse(url) # 현재 url을 파싱해서, 그 결과를 저장
        for p in parser_rss.entries:
            if p.title not in titles_rss: # 중복되지 않은 기사 일 경우 if 문 실행
                array_rss.append({'title':p.title, 'link':p.link})
                titles_rss.add(p.title)
#             else:
#                 print(f"Duplicated Title : {p.title}")
    return array_rss

# 링크를 통해 제목, 본문 내용 추출
def crawl_article(url, language='ko'):
    print(f"[Crawling. . .] {url}") # 현재 크롤링하는 링크 출력
    news_paper = Article(url, language=language)
    news_paper.download()
    news_paper.parse()
    return news_paper.title, preprocessing(news_paper.text)

# 불용어 전처리
def preprocessing(text): 
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그 제거하기
    return text_article

In [3]:
# 정보 수집할 rss 링크 모음
urls = ['http://rss.etnews.com/Section901.xml',
        'http://rss.etnews.com/Section902.xml',
        'http://rss.etnews.com/Section903.xml',
        'http://rss.etnews.com/Section904.xml']

# 제목과 기사 링크 수집하기
list_articles = crawl_rss(urls)

# 수집한 (제목,기사링크)를 토대로 제목과 본문 내용 수집하기
for article in list_articles:
    _, text = crawl_article(article['link'])
    article['text'] = text

[Crawl Rss] http://rss.etnews.com/Section901.xml
[Crawl Rss] http://rss.etnews.com/Section902.xml
[Crawl Rss] http://rss.etnews.com/Section903.xml
[Crawl Rss] http://rss.etnews.com/Section904.xml
[Crawling. . .] https://www.etnews.com/20230927000013
[Crawling. . .] https://www.etnews.com/20230926000368
[Crawling. . .] https://www.etnews.com/20230927000007
[Crawling. . .] https://www.etnews.com/20230926000363
[Crawling. . .] https://www.etnews.com/20230926000316
[Crawling. . .] https://www.etnews.com/20230926000189
[Crawling. . .] https://www.etnews.com/20230926000364
[Crawling. . .] https://www.etnews.com/20230926000344
[Crawling. . .] https://www.etnews.com/20230926000205
[Crawling. . .] https://www.etnews.com/20230926000340
[Crawling. . .] https://www.etnews.com/20230926000328
[Crawling. . .] https://www.etnews.com/20230926000101
[Crawling. . .] https://www.etnews.com/20230924000164
[Crawling. . .] https://www.etnews.com/20230926000288
[Crawling. . .] https://www.etnews.com/202309260

# 수집 내용 확인하기

In [4]:
list_articles[0]

{'title': '부산시 국가 R&D 예산 증액 분야에 전략 대응',
 'link': 'https://www.etnews.com/20230927000013',
 'text': "부산시가 정부 연구개발(R&D) 제도 변화에 발맞춰 부산 R&D 체계를 혁신한다.\n\n\n\n부산시는 최근 지역 내외 산·학·연·관 R&D 관계자 30여명과 '부산 R&D 혁신 정책토론회'을 열고, 정부 R&D 제도 변화에 따른 국비 유치 전략 재정립을 비롯해 R&D 혁신방안을 모색했다고 27일 밝혔다.\n\n\n\n토론회 참석자들은 부산 R&D 혁신 방안으로 △R&D체계 개선 △12대 국가전략기술 부산 유망 분야 집중 육성 △지역전략산업 글로벌 초격차 R&D 추진 △부산 R&D기획 지원제도 개편 △부산 자체 R&D 디지털플랫폼 마련 등을 제시했다.\n\n\n\nR&D체계 개선을 위해서는 정부 7개 핵심기술 분야 지역 공동기획·협력사업에 대한 가산점 부여와 상대평가 방식의 성과평가 도입을 제안했다.\n\n\n\n12대 국가전략기술 가운데 부산 유망 분야를 선정해 기초 원천기술 개발에서 산업 생태계 조성까지 집중 지원해야 한다는 의견도 나왔다. 지역 핵심 거점과 연계한 지역전략산업 초격차 R&D 추진 필요성도 제시됐다.\n\n\n\n참석자들은 특히 양자, 이차전지, 첨단바이오, 반도체 등 국가 R&D사업 예산이 증액될 분야의 공모사업에 전략적 대응이 필요하다고 입을 모았다. 국비 유치를 위해 산학연관 협력체계와 R&D 기획력 강화를 주문했다.\n\n\n\n이경덕 부산시 미래산업국장은 “제시된 의견을 수렴해 구체적인 R&D 혁신 방향과 방안을 마련하겠다”며 “국가 R&D 변화에 선도적으로 대응해 신규 R&D 분야에서 국비를 대폭 확보하는 성과를 거두겠다”고 말했다.\n\n\n\n한편, 정부는 선도형 R&D로 전환을 위해 '정부 R&D 제도혁신 방안'을 수립하고 지난 8월 이를 반영한 '2024년 국가 R&D 예산 배분 조정결과'를 발표했다.\n\n\n\n정부 R&D 제도혁신 방안은 정부

# 모든 본문 text에서 명사  (키워드, 빈도수) 추출

##### 키워드와 빈도수 추출하는 함수 정의

In [5]:
def get_keywords_and_frequency(text,nKeywords=10): #텍스트와  추출할 키워드 개수
    spliter = Okt() # Okt 객체 생성
    nouns = spliter.nouns(text) # 명사 추출 
    count = Counter(nouns) # 명사별 빈도수 추출
    list_keywords = []
    
    for n,c in count.most_common(nKeywords): # 최빈값 높은 명사 순으로, nKeywords까지 출력
        item = {'keyword':n, 'count':c}
        list_keywords.append(item)
    return list_keywords

##### 추출 내용 저장하기

In [6]:
for article in list_articles:
    keywords = get_keywords_and_frequency(article['text'])
    article['keywords'] = keywords

# 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력하기

In [7]:
# 검색어와 동일한 키워드가 있을 때, 빈도수 반환
def search_articles(query, list_keywords): # 쿼리가 키워드 리스트에 있으면 출력 (없으면 0)
    nWords = 0 # 키워드 빈도수 초기화
    for kw in list_keywords:
        if eq(query, kw['keyword']):
            nWords = kw['count']
    return nWords

In [10]:
query = input("키워드를 입력하세요 : ")

for article in list_articles:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        print(f"[TF] {nQuery}, [Title] {article['title']}, [URL] {article['link']}")

키워드를 입력하세요 : 반도체
[TF] 6, [Title] 삼성·LG, 강도 높은 재고조정 효과...3분기 실적 개선 기대↑, [URL] https://www.etnews.com/20230926000178
[TF] 20, [Title] 전자신문 테크서밋 내달 18일 개막…반도체·디스플레이 미래기술 엿본다, [URL] https://www.etnews.com/20230926000282
[TF] 25, [Title] "中서 현상 유지만 하라는 것…첨단 반도체 대규모 투자 어려워", [URL] https://www.etnews.com/20230924000040
[TF] 8, [Title] 美, '반도체법' 가드레일 규정 최종 발표, [URL] https://www.etnews.com/20230923000006
[TF] 6, [Title] 펨트론, 웨이퍼 표면 검사 장비 국산화, [URL] https://www.etnews.com/20230925000191


In [9]:
list_articles[0]

{'title': '부산시 국가 R&D 예산 증액 분야에 전략 대응',
 'link': 'https://www.etnews.com/20230927000013',
 'text': "부산시가 정부 연구개발(R&D) 제도 변화에 발맞춰 부산 R&D 체계를 혁신한다.\n\n\n\n부산시는 최근 지역 내외 산·학·연·관 R&D 관계자 30여명과 '부산 R&D 혁신 정책토론회'을 열고, 정부 R&D 제도 변화에 따른 국비 유치 전략 재정립을 비롯해 R&D 혁신방안을 모색했다고 27일 밝혔다.\n\n\n\n토론회 참석자들은 부산 R&D 혁신 방안으로 △R&D체계 개선 △12대 국가전략기술 부산 유망 분야 집중 육성 △지역전략산업 글로벌 초격차 R&D 추진 △부산 R&D기획 지원제도 개편 △부산 자체 R&D 디지털플랫폼 마련 등을 제시했다.\n\n\n\nR&D체계 개선을 위해서는 정부 7개 핵심기술 분야 지역 공동기획·협력사업에 대한 가산점 부여와 상대평가 방식의 성과평가 도입을 제안했다.\n\n\n\n12대 국가전략기술 가운데 부산 유망 분야를 선정해 기초 원천기술 개발에서 산업 생태계 조성까지 집중 지원해야 한다는 의견도 나왔다. 지역 핵심 거점과 연계한 지역전략산업 초격차 R&D 추진 필요성도 제시됐다.\n\n\n\n참석자들은 특히 양자, 이차전지, 첨단바이오, 반도체 등 국가 R&D사업 예산이 증액될 분야의 공모사업에 전략적 대응이 필요하다고 입을 모았다. 국비 유치를 위해 산학연관 협력체계와 R&D 기획력 강화를 주문했다.\n\n\n\n이경덕 부산시 미래산업국장은 “제시된 의견을 수렴해 구체적인 R&D 혁신 방향과 방안을 마련하겠다”며 “국가 R&D 변화에 선도적으로 대응해 신규 R&D 분야에서 국비를 대폭 확보하는 성과를 거두겠다”고 말했다.\n\n\n\n한편, 정부는 선도형 R&D로 전환을 위해 '정부 R&D 제도혁신 방안'을 수립하고 지난 8월 이를 반영한 '2024년 국가 R&D 예산 배분 조정결과'를 발표했다.\n\n\n\n정부 R&D 제도혁신 방안은 정부